In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud as wc

In [2]:
data = pd.read_csv("data/proccessed_data.csv")
data.head()

,Unnamed: 0,id,party_name,descriptions,gender,is_foreigner,location,descr_procc
0,0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,hasič;Všeruby (Domažlice);ex-zast_obec;ex-kand...,1,2,554456,"['hasič', 'všeruby', '(domažlice)', 'ex-zastup..."
1,1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0,"['potravinářský', 'komora', 'český', 'republik..."
2,2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0,"['umělec', 'dirigent']"
3,3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0,"['umělec', 'básnířka']"
4,4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a ...,2,2,0,"['mluvčí', 'rožnov', 'radhoštěm', 'umělec', 'g..."


In [3]:
data.drop(columns=['Unnamed: 0'], inplace=True)
data.head()

,id,party_name,descriptions,gender,is_foreigner,location,descr_procc
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,hasič;Všeruby (Domažlice);ex-zast_obec;ex-kand...,1,2,554456,"['hasič', 'všeruby', '(domažlice)', 'ex-zastup..."
1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0,"['potravinářský', 'komora', 'český', 'republik..."
2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0,"['umělec', 'dirigent']"
3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0,"['umělec', 'básnířka']"
4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a ...,2,2,0,"['mluvčí', 'rožnov', 'radhoštěm', 'umělec', 'g..."


In [4]:
import ast


data.descr_procc = data.descr_procc.apply(ast.literal_eval)

In [5]:
data.descr_procc[1]

['potravinářský', 'komora', 'český', 'republika', 'lobby', 'mluvčí']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
bag = vec.fit_transform(data.descr_procc.apply(lambda x: ' '.join(x)))
vec.get_feature_names_out()

/home/Ales/Programms/miniconda3/envs/rcluster/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array(['aaa', 'aaabyty', 'aag', ..., 'žš', 'žžm', 'ˇyďár'], dtype=object)

In [7]:
bag_of_words=pd.DataFrame.sparse.from_spmatrix(bag,columns=vec.get_feature_names_out())
bag_of_words.head()

,aaa,aaabyty,aag,aakrchitektura,aalborg,aarhus,ab,abatyše,abb,abc,...,žďár,žďárce,žďárku,žďársko,žďársku,žďárský,žďáře,žš,žžm,ˇyďár
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
bag_of_words.sum().sort_values(ascending=False).head(50)

kandidát         16835
ex               15818
odborník         11387
obec             11323
sport             9604
zastupitel        8923
umělec            8756
reg               8178
zpr               8173
kultura           7330
byznys            7180
ředitel           6387
posl              5836
mluvčí            5710
spolek            4671
starosta          4574
vedoucí           4053
akademik          3734
zdravotník        3523
úředník           3358
novinář           3205
český             3035
předseda          2610
ředitelka         2563
město             2157
kraj              1943
ido               1866
om                1865
social            1819
ozbrojenec        1749
čr                1714
trenér            1632
společnost        1592
krajský           1558
ústav             1451
bývalý            1433
muzeum            1414
církev            1411
místostarosta     1373
politik           1340
univerzita        1255
odbor             1236
fotbalista        1232
hokejista  

In [9]:
from sklearn.cluster import KMeans
best_clusters = 0
best_score = 0
for i in range(2, 10):
    km = KMeans(n_clusters=i)
    km.fit(bag_of_words)
    score = km.score(bag_of_words)
    if score > best_score:
        best_score = score
        best_clusters = i
    print(i, score)

MemoryError: Unable to allocate 17.0 GiB for an array with shape (28012, 81625) and data type int64